## Step 1: Load Libraries

In [2]:
import scanpy as sc
import numpy as np
import scipy
import pandas as pd
from scipy.io import mmwrite
import argparse
import os

## Step 2: Load Library 1 Data

In [3]:
file_load =  "/gstore/scratch/u/ghaffars/glmGamPoi/sublib1_bdev/data/remove_pos_cont_counts_obs_var.h5ad"
data = sc.read_h5ad(file_load)

## Step 3: Obtain Column Meta Data

In [4]:
## Create NTC vs not label
data.obs["label"] = ['aNTC' if gene_symbol == "NTC" else 'zAll' for gene_symbol in data.obs['gene_symbol']]

In [5]:
data.obs['gem'] = data.obs['NGS_ID'].astype(str) + '-' + data.obs['10Xrun'].astype(str)

In [6]:
## Meta Data for pyGamPoi Model
meta = pd.DataFrame({'label':data.obs["label"],
                   'cell':data.obs.index,
                   'total_counts':data.obs["total_counts"],
                   'fct_counts_mt':data.obs["pct_counts_mt"]/100,
                   'batchid':data.obs["gem"],
                   'guidename':data.obs["DemuxAssignment_crispr"],
                   'targetname':data.obs["gene_symbol"]})

In [7]:
## Change guidename to not have commas to not confuse bash later
meta["guidename"] = meta["guidename"].str.replace(",", ":")


In [ ]:
## File to save      
meta_file  = "/gstore/scratch/u/ghaffars/glmGamPoi/sublib1_bdev/data/coldata_for_glmgampois.csv"
## Save
meta.to_csv(meta_file, index = False)

In [ ]:
## Save row data
full_gene_file  = "/gstore/scratch/u/ghaffars/glmGamPoi/sublib1_bdev/data/full_gene_data.csv"
data.var.to_csv(full_gene_file, index = False)

In [8]:
data

AnnData object with n_obs × n_vars = 595042 × 36603
    obs: 'Sample', 'Barcode', 'DemuxType_crispr', 'DemuxAssignment_crispr', 'DemuxType_hashing', 'DemuxAssignment_hashing', 'cellline', 'timepoint', 'HTO', 'NGS_ID', 'Biological_replicate', '10Xrun', 'sublibrary', 'gRNA_library_MOI', 'gene_symbol', 'class', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'qc_pass', 'S_score', 'G2M_score', 'phase', 'label', 'gem'
    var: 'Symbol'
    layers: 'counts'

## Step 4: Split and Save by Target

In [ ]:
## List of targets
targets = data.obs["gene_symbol"].unique()

## Create Folder To Save by Target
by_target_folder =  "/gstore/scratch/u/ghaffars/glmGamPoi/sublib1_bdev/data/by_target/"

## Ensure folder exists
if not os.path.exists(by_target_folder):
    os.makedirs(by_target_folder)

## Save info about column names of top 3000 genes
print("length target is " + str(len(targets)))

## Iterate over target and save    
for i in range(len(targets)): 
    if i % 100 == 0:
            print(i)
    target = targets[i]
    data_target = data[data.obs["gene_symbol"].isin([target,"NTC"])].copy()
    data_target.X = data_target.X.astype("float32")
    data_target.layers['counts'] = data_target.layers['counts'].astype("float32")
    ## Save as .h5ad
    data_target.write(by_target_folder+target+".h5ad")
    

print("filtered and saved by target")
print("all done")


      

length target is 4975
0
100
200
300
400
500
600
700
